# 找到问题了, 是normalize的问题!

In [3]:
import argparse
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data.dataloader import DataLoader
# import utils.data.functions
import os, sys, os.path
import torch.nn as nn

D:\XAPP\miniconda3\envs\ai\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [4]:
from torch.utils.data import random_split
from pytorch_lightning.callbacks import EarlyStopping

In [5]:
import torchmetrics

In [6]:
sys.path.append(r'D:\recRoot\GitRepos\~external\T-GCN\T-GCN\T-GCN-PyTorch')

In [7]:
import utils.callbacks

# DataLoader

In [8]:
class TGCN_dataset(torch.utils.data.Dataset):
    def __init__(self):
        for f in self.processed_file_names:
            if not os.path.exists(f):
                self.process()
        self.data = torch.load(self.processed_file_names[0])

    @property
    def raw_file_names(self):
        return ['data/sz_adj.csv', 'data/sz_speed.csv']

    @property
    def processed_file_names(self):
        return ['data_sz_GRU.pt']

    def download(self):
        # Download to `self.raw_dir`.
        print("Please download the data manually!")
        sys.exit(0)
        
    def process(self):
        for f in self.raw_file_names:
            if not os.path.exists(f):
                slef.download()
                break
        print("Processing ... ...")
        # Read data into huge `Data` list.
        fn_adj, fn_spd = self.raw_file_names
        df_spd = pd.read_csv(fn_spd)
        S = df_spd.values  # nTimes x nNodes
        
        data = np.zeros((S.shape[0]-14, 15, S.shape[1]))
        for i in range(data.shape[0]):
            data[i,:,:] = S[i:i+15, :]
        
        torch.save(torch.Tensor(data), self.processed_file_names[0])
        print("Process done.")
        
    def clean(self):
        os.remove('data_sz_GRU.pt')
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx: int):
        return self.data[idx, :, :]

In [9]:
ds = TGCN_dataset()

ratio_valid = 0.2

size_valid = int(ratio_valid * len(ds))
size_train = len(ds) - size_valid
ds_learn, ds_valid = random_split(ds, [size_train, size_valid])

def collate_fn(batch):
#     global b1
#     b1 = batch
    # batch: (batchSize, seq, nFeatures)
    batchTS = torch.stack(batch)
    return batchTS[:, :12, :], batchTS[:, 12:, :].permute(0, 2, 1).contiguous()
#     return batchTS[:, :12, :].permute(0,2,1).contiguous(), batchTS[:, 12:, :].permute(0,2,1).contiguous()# .view(batchTS.shape[0], -1)

dlr_train = torch.utils.data.DataLoader(ds_learn, shuffle=True, batch_size=64, collate_fn=collate_fn)

dlr_valid = torch.utils.data.DataLoader(ds_valid, shuffle=False, batch_size=size_valid, collate_fn=collate_fn)

In [10]:
a, b = next(iter(dlr_train))
print(a.shape)
print(b.shape)

torch.Size([64, 12, 156])
torch.Size([64, 156, 3])


In [11]:

class GRULinear(nn.Module):
    def __init__(self, num_gru_units: int, output_dim: int, bias: float = 0.0):
        super(GRULinear, self).__init__()
        self._num_gru_units = num_gru_units
        self._output_dim = output_dim
        self._bias_init_value = bias
        self.weights = nn.Parameter(
            torch.FloatTensor(self._num_gru_units + 1, self._output_dim)
        )
        self.biases = nn.Parameter(torch.FloatTensor(self._output_dim))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weights)
        nn.init.constant_(self.biases, self._bias_init_value)

    def forward(self, inputs, hidden_state):
        batch_size, num_nodes = inputs.shape
        # inputs (batch_size, num_nodes, 1)
        inputs = inputs.reshape((batch_size, num_nodes, 1))
        # hidden_state (batch_size, num_nodes, num_gru_units)
        hidden_state = hidden_state.reshape(
            (batch_size, num_nodes, self._num_gru_units)
        )
        # [inputs, hidden_state] "[x, h]" (batch_size, num_nodes, num_gru_units + 1)
        concatenation = torch.cat((inputs, hidden_state), dim=2)
        # [x, h] (batch_size * num_nodes, gru_units + 1)
        concatenation = concatenation.reshape((-1, self._num_gru_units + 1))
        # [x, h]W + b (batch_size * num_nodes, output_dim)
        outputs = concatenation @ self.weights + self.biases
        # [x, h]W + b (batch_size, num_nodes, output_dim)
        outputs = outputs.reshape((batch_size, num_nodes, self._output_dim))
        # [x, h]W + b (batch_size, num_nodes * output_dim)
        outputs = outputs.reshape((batch_size, num_nodes * self._output_dim))
        return outputs

    def hyperparameters(self):
        return {
            "num_gru_units": self._num_gru_units,
            "output_dim": self._output_dim,
            "bias_init_value": self._bias_init_value,
        }


class GRUCell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int):
        super(GRUCell, self).__init__()
        self._input_dim = input_dim
        self._hidden_dim = hidden_dim
        self.linear1 = GRULinear(self._hidden_dim, self._hidden_dim * 2, bias=1.0)
        self.linear2 = GRULinear(self._hidden_dim, self._hidden_dim)

    def forward(self, inputs, hidden_state):
        # [r, u] = sigmoid([x, h]W + b)
        # [r, u] (batch_size, num_nodes * (2 * num_gru_units))
        concatenation = torch.sigmoid(self.linear1(inputs, hidden_state))
        # r (batch_size, num_nodes * num_gru_units)
        # u (batch_size, num_nodes * num_gru_units)
        r, u = torch.chunk(concatenation, chunks=2, dim=1)
        # c = tanh([x, (r * h)]W + b)
        # c (batch_size, num_nodes * num_gru_units)
        c = torch.tanh(self.linear2(inputs, r * hidden_state))
        # h := u * h + (1 - u) * c
        # h (batch_size, num_nodes * num_gru_units)
        new_hidden_state = u * hidden_state + (1 - u) * c
        return new_hidden_state, new_hidden_state

    @property
    def hyperparameters(self):
        return {"input_dim": self._input_dim, "hidden_dim": self._hidden_dim}


class GRU(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, **kwargs):
        super(GRU, self).__init__()
        self._input_dim = input_dim  # num_nodes for prediction
        self._hidden_dim = hidden_dim
        self.gru_cell = GRUCell(self._input_dim, self._hidden_dim)

    def forward(self, inputs):
        batch_size, seq_len, num_nodes = inputs.shape
        assert self._input_dim == num_nodes
        outputs = list()
        hidden_state = torch.zeros(batch_size, num_nodes * self._hidden_dim).type_as(
            inputs
        )
        for i in range(seq_len):
            output, hidden_state = self.gru_cell(inputs[:, i, :], hidden_state)
            output = output.reshape((batch_size, num_nodes, self._hidden_dim))
            outputs.append(output)
        last_output = outputs[-1]
        return last_output


    @property
    def hyperparameters(self):
        return {"input_dim": self._input_dim, "hidden_dim": self._hidden_dim}


In [12]:

class SupervisedForecastTask(pl.LightningModule):
    def __init__(
        self,
        model: nn.Module,
        pre_len: int = 3,
        learning_rate: float = 1e-3,
        weight_decay: float = 1.5e-3,
        **kwargs
    ):
        super(SupervisedForecastTask, self).__init__()
        self.save_hyperparameters(ignore=['model'])
        self.model = model
        self.regressor = nn.Linear(
                self.model.hyperparameters.get("hidden_dim")
                or self.model.hyperparameters.get("output_dim"),
                self.hparams.pre_len,
            )
        self.lossesV = []

    def forward(self, x):
        # (batch_size, seq_len, num_nodes)
        batch_size, _, num_nodes = x.size()
        # (batch_size, num_nodes, hidden_dim)
        hidden = self.model(x)
        # (batch_size * num_nodes, hidden_dim)
        hidden = hidden.reshape((-1, hidden.size(2)))
        # (batch_size * num_nodes, pre_len)
        predictions = self.regressor(hidden)
        
        predictions = predictions.reshape((batch_size, num_nodes, -1))
#         print(predictions.shape)
        return predictions

    def shared_step(self, batch, batch_idx):
        # (batch_size, seq_len/pre_len, num_nodes)
        x, y = batch
        num_nodes = x.size(2)
        predictions = self(x)
#         predictions = predictions.transpose(1, 2).reshape((-1, num_nodes))
#         y = y.reshape((-1, y.size(2)))
        return predictions, y

    def loss(self, inputs, targets):
#         print(inputs.shape)
#         print(targets.shape)
        return torch.nn.functional.mse_loss(inputs, targets)
#         if self._loss == "mse":
#             return F.mse_loss(inputs, targets)
#         if self._loss == "mse_with_regularizer":
#             return utils.losses.mse_with_regularizer_loss(inputs, targets, self)
#         raise NameError("Loss not supported:", self._loss)

    def training_step(self, batch, batch_idx):
        predictions, y = self.shared_step(batch, batch_idx)
        loss = self.loss(predictions, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        predictions, y = self.shared_step(batch, batch_idx)
#         predictions = predictions * self.feat_max_val
#         y = y * self.feat_max_val
        loss = self.loss(predictions, y)
        self.lossesV.append(loss)
        rmse = torch.sqrt(torchmetrics.functional.mean_squared_error(predictions, y))
#         mae = torchmetrics.functional.mean_absolute_error(predictions, y)
#         accuracy = utils.metrics.accuracy(predictions, y)
#         r2 = utils.metrics.r2(predictions, y)
#         explained_variance = utils.metrics.explained_variance(predictions, y)
        metrics = {
            "val_loss": loss,
            "RMSE": rmse,
#             "MAE": mae,
#             "accuracy": accuracy,
#             "R2": r2,
#             "ExplainedVar": explained_variance,
        }
        self.log_dict(metrics)
        return predictions.reshape(batch[1].size()), y.reshape(batch[1].size())

    def test_step(self, batch, batch_idx):
        pass

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.parameters(),
            lr=self.hparams.learning_rate,
            weight_decay=self.hparams.weight_decay,
        )


    def train_dataloader(self):
        return dlr_train

    def val_dataloader(self):
        return dlr_valid

In [13]:
def get_callbacks():
    early_stop_callback = EarlyStopping(
        monitor='val_loss',  # change to 'train_loss' or any other metric you want to monitor
        min_delta=0.00,
        patience=100,
        verbose=False,
        mode='min'
    )
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="train_loss")
    plot_validation_predictions_callback = utils.callbacks.PlotValidationPredictionsCallback(monitor="train_loss")
    callbacks = [
        checkpoint_callback,
        plot_validation_predictions_callback,
        early_stop_callback
    ]
    return callbacks



In [14]:
trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=2000, callbacks=get_callbacks())

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
net = GRU(156, 100)

In [16]:
task = SupervisedForecastTask(model=net)

In [17]:
trainer.fit(task)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type   | Params
-------------------------------------
0 | model     | GRU    | 30.6 K
1 | regressor | Linear | 303   
-------------------------------------
30.9 K    Trainable params
0         Non-trainable params
30.9 K    Total params
0.124     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|                                                              | 0/1 [00:00<?, ?it/s]

D:\XAPP\miniconda3\envs\ai\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


D:\XAPP\miniconda3\envs\ai\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\XAPP\miniconda3\envs\ai\Lib\site-packages\pytorch_lightning\trainer\trainer.py:1609: PossibleUserWarning: The number of training batches (38) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 44.80it/s, loss=220, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 1:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 46.58it/s, loss=165, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 2:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 46.02it/s, loss=126, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 3:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 46.30it/s, loss=112, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 4:  97%|███████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 45.31it/s, loss=92.9, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 5:  97%|███████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.61it/s, loss=79.1, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 6:  97%|███████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.61

Epoch 16:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 47.36it/s, loss=44, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 17:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 47.45it/s, loss=43, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 18:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.10it/s, loss=42.5, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 19:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 47.10it/s, loss=40, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 20:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.64it/s, loss=38.8, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 21:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.75it/s, loss=38.9, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 22:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.95

Epoch 31:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.33it/s, loss=33.4, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 32:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.72it/s, loss=32.6, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 33:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.39it/s, loss=33.3, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 34:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.78it/s, loss=31.3, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 35:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.58it/s, loss=32.3, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 36:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 46.69it/s, loss=32, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 37:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.16

Epoch 47:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.04it/s, loss=30.2, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 48:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.04it/s, loss=31.6, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 49:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.89it/s, loss=30.6, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 50:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 39.63it/s, loss=30.6, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 51:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 44.56it/s, loss=30.4, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 52:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.36it/s, loss=31.4, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 53:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.04

Epoch 62:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.57it/s, loss=31.5, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 63:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 47.01it/s, loss=28, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 64:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.13it/s, loss=30.6, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 65:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.21it/s, loss=30.4, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 66:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 46.61it/s, loss=30.3, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 67:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.18it/s, loss=28.2, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 68:  97%|████████████████████████████████████████████████████▌ | 38/39 [00:00<00:00, 46.

Epoch 78:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 45.47it/s, loss=29.5, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 79:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.36it/s, loss=28.7, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 80:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.69it/s, loss=30.2, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 81:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.69it/s, loss=28.2, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 82:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.84it/s, loss=28.9, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 83:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.21it/s, loss=28.3, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 84:  97%|██████████████████████████████████████████████████▋ | 38/39 [00:00<00:00, 47.42

In [ ]:
# results = trainer.validate()

In [2]:
task.lossesV

NameError: name 'task' is not defined

In [1]:
xV, yV = next(iter(dlr_valid))

NameError: name 'dlr_valid' is not defined